In [2]:
import astropy.io.fits as pf
import os
import sys
import numpy as np
import datetime

In [ ]:
import pyds9

In [ ]:
ds = pyds9.DS9()

In [38]:
def check_header_refpoint(hdr):
    """
    Check a FITS header to see if it does not have the expected data table
    columns for telescope aspect.
    
    Require the following keywords:
    
    TELESCOP = 'NuSTAR'
    INSTRUME = 'FPM?'           # ? is 'A' or 'B'
    EXTNAME  = 'DET?_REFPOINT'  # ? is a digit 1 to 4
    HDUCLAS1 = 'TEMPORALDATA'
    NAXIS    = 2
    
    Require the following fields:
    
    TTYPE1   = 'TIME'
    TUNIT1   = 's'
    TTYPE2   = 'X_DET1'
    TUNIT2   = 'pixel'
    TTYPE3   = 'Y_DET1'
    TUNIT3   = 'pixel'
    """
    try:
        if not (
            hdr['TELESCOP'] == 'NuSTAR' and
            (hdr['INSTRUME'] == 'FPMA' or hdr['INSTRUME'] == 'FPMB') and
            (hdr['EXTNAME'] in
                ['DET%d_REFPOINT' % i for i in (1, 2, 3, 4)]) and
            hdr['HDUCLAS1'] == 'TEMPORALDATA' and
            hdr['NAXIS'] == 2 and
            hdr['TTYPE1'] == 'TIME' and hdr['TUNIT1'] == 's' and
            (hdr['TTYPE2'] in
                ['X_DET%d' % i for i in (1, 2, 3, 4)]) and
            hdr['TUNIT2'] == 'pixel' and
            (hdr['TTYPE3'] in
                ['Y_DET%d' % i for i in (1, 2, 3, 4)]) and
            hdr['TUNIT3'] == 'pixel'
        ):
            return False
    except KeyError:
        return False

    return True

In [45]:
def check_header_gti(hdr):
    """
    Check FITS header to see if it has the expected data table columns for GTI extension.
    
    Require the following keywords:
    
    TELESCOP = 'NuSTAR'
    EXTNAME  = 'STDGTI'  # ? is a digit 1 to 4
    NAXIS    = 2
    
    Require the following fields:
    
    TTYPE1   = 'START'
    TUNIT1   = 's' or 'sec'
    TTYPE2   = 'STOP'
    TUNIT2   = 's' or 'sec'
    """
    try:
        if not (
            hdr['TELESCOP'] == 'NuSTAR' and
            hdr['EXTNAME'] == 'STDGTI' and
            hdr['NAXIS'] == 2 and
            hdr['TTYPE1'] == 'START' and
            hdr['TUNIT1'] in ('s', 'sec') and
            hdr['TTYPE2'] == 'STOP' and
            hdr['TUNIT2'] in ('s', 'sec')
        ):
            return False
    except KeyError:
        return False

    return True

In [4]:
os.chdir('/Users/qw/astro/nustar/IC342_X1/90201039002/event_cl')
os.getcwd()

'/Users/qw/astro/nustar/IC342_X1/90201039002/event_cl'

In [28]:
check_header_refpoint(detB1fh[1].header)

True

In [32]:
refpointfile = 'nu90201039002B_det1.fits'
detB1fh = pf.open(refpointfile)
detB1fh.info()

Filename: nu90201039002B_det1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      40   ()      
  1  DET1_REFPOINT    1 BinTableHDU     75   389507R x 3C   [1D, 1D, 1D]   


In [34]:
refpointext = None
for ext in detB1fh:
    if check_header_refpoint(ext.header):
        refpointext = ext
        break
if refpointext is None:
    print('No aspect info in the specified file %s.' % refpointfile)
else:
    print('Found extension %s.' % refpointext.header['EXTNAME'])

Found extension DET1_REFPOINT.


In [37]:
refpointext.data.columns

ColDefs(
    name = 'TIME'; format = '1D'; unit = 's'
    name = 'X_DET1'; format = '1D'; unit = 'pixel'
    name = 'Y_DET1'; format = '1D'; unit = 'pixel'
)

In [42]:
refpointext.data

FITS_rec([(2.14279802e+08, 345.94213321, 390.56390939),
          (2.14279802e+08, 345.9289652 , 390.29421492),
          (2.14279802e+08, 345.81048348, 390.35291152), ...,
          (2.14368601e+08, 384.12522513, 366.16993093),
          (2.14368601e+08, 383.98007602, 365.28113636),
          (2.14368601e+08, 382.41689515, 366.00035761)],
         dtype=(numpy.record, [('TIME', '>f8'), ('X_DET1', '>f8'), ('Y_DET1', '>f8')]))

In [40]:
gtifile = 'nu90201039002B01_gti.fits'
gtiBfh = pf.open(gtifile)
gtiBfh.info()

Filename: nu90201039002B01_gti.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      72   ()      
  1  STDGTI        1 BinTableHDU     89   40R x 2C   [1D, 1D]   


In [46]:
gtiext = None
for ext in gtiBfh:
    if check_header_gti(ext.header):
        gtiext = ext
        break
if gtiext is None:
    print('No GTI info in the specified file %s.' % gtifile)
else:
    print('Found extension %s.' % gtiext.header['EXTNAME'])

Found extension STDGTI.


In [74]:
gtiext.data[:10]

FITS_rec([(2.14280304e+08, 2.14283833e+08),
          (2.14283835e+08, 2.14283835e+08),
          (2.14286105e+08, 2.14289627e+08),
          (2.14289628e+08, 2.14289629e+08),
          (2.14289631e+08, 2.14289631e+08),
          (2.14291905e+08, 2.14295426e+08),
          (2.14297705e+08, 2.14300301e+08),
          (2.14300737e+08, 2.14301225e+08),
          (2.14301225e+08, 2.14301225e+08),
          (2.14301227e+08, 2.14301227e+08)],
         dtype=(numpy.record, [('START', '>f8'), ('STOP', '>f8')]))

In [70]:
gtiarr = np.sort(gtiext.data, order='START', kind='mergesort')

In [71]:
gtiarr[:10]

FITS_rec([(2.14280304e+08, 2.14283833e+08),
          (2.14283835e+08, 2.14283835e+08),
          (2.14286105e+08, 2.14289627e+08),
          (2.14289628e+08, 2.14289629e+08),
          (2.14289631e+08, 2.14289631e+08),
          (2.14291905e+08, 2.14295426e+08),
          (2.14297705e+08, 2.14300301e+08),
          (2.14300737e+08, 2.14301225e+08),
          (2.14301225e+08, 2.14301225e+08),
          (2.14301227e+08, 2.14301227e+08)],
         dtype=(numpy.record, [('START', '>f8'), ('STOP', '>f8')]))

In [72]:
refpointarr = np.sort(refpointext.data, order='TIME', kind='mergesort')

In [170]:
refpointarr[:10][0]

(214279802.02499428, 345.94213321250015, 390.56390938823506)

In [81]:
i_refpoint = 0
n_refpoint = len(refpointarr)
i_gti = 0
n_gti = len(gtiarr)
coords = []
dt = []

print('Processing %d aimpoints...' % n_refpoint)

while i_refpoint < (n_refpoint - 1) and i_gti < n_gti:
    # Ref pointings have a single time while GTI intervals have two.
    # Original logic in projobs.pro considers pointing time against [start, stop),
    # i.e. >= for start time, < for stop.
    
    if refpointarr[i_refpoint][0] < gtiarr[i_gti][0]:
        # Pointing is before current interval, go to next pointing
        i_refpoint += 1
        continue
    
    if refpointarr[i_refpoint][0] >= gtiarr[i_gti][1]:
        # Pointing is after current interval, go to next interval
        i_gti += 1
        continue
    
    # Otherwise there is some overlap. Add this pointing only if it is entirely within the GTI interval (original behavior).
    coords.append([refpointarr[i_refpoint][1], refpointarr[i_refpoint][2]])
    dt.append(refpointarr[i_refpoint + 1][0] - refpointarr[i_refpoint][0])
    i_refpoint += 1

Processing 389507 aimpoints...


In [86]:
print('%d aimpoints (%f / %f s)' % (
    len(coords), np.sum(dt),
    np.sum(gtiarr.field('STOP') - gtiarr.field('START'))))

209458 aimpoints (52483.125078 / 52483.125077 s)


In [100]:
asphistimg = np.zeros((1000, 1000), dtype=np.float64)
x_min, y_min, x_max, y_max = 999, 999, 0, 0
for i in range(len(dt)):
    x, y = int(np.floor(coords[i][0])), int(np.floor(coords[i][1]))
    asphistimg[y, x] += dt[i]
    if y < y_min:
        y_min = y
    elif y > y_max:
        y_max = y
    if x < x_min:
        x_min = x
    elif x > x_max:
        x_max = x

In [101]:
ds.set_np2arr(asphistimg[y_min:y_max+1, x_min:x_max+1])

1

In [177]:
print('Image subspace: ', x_min, x_max, y_min, y_max)

Image subspace:  365 399 344 390


In [103]:
np.sum(asphistimg[y_min:y_max+1, x_min:x_max+1])

52483.12507840991

In [104]:
np.sum(asphistimg)

52483.12507840991

In [175]:
aspecthistfh = pf.HDUList(
    pf.PrimaryHDU(asphistimg[y_min:y_max+1, x_min:x_max+1])
)
aspecthistfh[0].header['EXTNAME'] = 'ASPECT_HISTOGRAM'
aspecthistfh[0].header['X_OFF'] = (x_min, 'x offset in pixels')
aspecthistfh[0].header['Y_OFF'] = (y_min, 'y offset in pixels')
aspecthistfh[0].header['EXPOSURE'] = (np.float32(np.sum(asphistimg[y_min:y_max+1, x_min:x_max+1])), 'seconds, total exposure time')
aspecthistfh[0].header['COMMENT'] = 'Add the x/y offset to image coordinates to recover aimpoint in detector coordinates.'
aspecthistfh[0].header['DATE'] = (datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S'), 'File creation date (YYYY-MM-DDThh:mm:ss UTC)')
aspecthistfh[0].header['HISTORY'] = 'Aspect histogram image created by filtering %s using %s.' % (
    refpointfile, gtifile
)
for keyword in ('TELESCOP', 'INSTRUME', 'OBS_ID', 'OBJECT', 'TARG_ID', 
                'RA_OBJ', 'DEC_OBJ', 'RA_NOM', 'DEC_NOM', 'RA_PNT', 'DEC_PNT', 'TIMESYS',
                'MJDREFI', 'MJDREFF', 'CLOCKAPP', 'TSTART', 'TSTOP', 'DATE-OBS', 'DATE-END'):
    if keyword in refpointext.header:
        aspecthistfh[0].header[keyword] = (refpointext.header[keyword], refpointext.header.comments[keyword])


In [176]:
aspecthistfh[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   35                                                  
NAXIS2  =                   47                                                  
EXTEND  =                    T                                                  
EXTNAME = 'ASPECT_HISTOGRAM'                                                    
X_OFF   =                  365 / x offset in pixels                             
Y_OFF   =                  344 / y offset in pixels                             
EXPOSURE=            52483.125 / seconds, total exposure time                   
DATE    = '2019-08-09T12:10:17' / File creation date (YYYY-MM-DDThh:mm:ss UTC)  
TELESCOP= 'NuSTAR  '           / Telescope (mission) name                       
INSTRUME= 'FPMB    '        

In [137]:
aspecthistfh.writeto('aspecthistB.fits')

In [139]:
refpointext.header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                   24 / width of table in bytes                        
NAXIS2  =               389507 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                    3 / number of fields in each row                   
TTYPE1  = 'TIME    '           / Event Time (seconds since Jan 2010 00:00:00 UTC
TFORM1  = '1D      '           / data format of the field: DOUBLE PRECISION     
TUNIT1  = 's       '           / physical unit of field                         
TTYPE2  = 'X_DET1  '           / Detector Reference Point X position (SKY Frame)
TFORM2  = '1D      '        